In [1]:
from google.colab import drive
drive.mount("/content/drive")
modelPath = "/content/drive/My Drive/ambulance_prediction"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from imutils import paths

Using TensorFlow backend.


In [3]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = "/content/drive/My Drive/ambulance_prediction/data/train"
validation_data_dir = "/content/drive/My Drive/ambulance_prediction/data/validation"
nb_train_samples = len(list(paths.list_images(train_data_dir)))
nb_validation_samples = len(list(paths.list_images(validation_data_dir)))
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
  input_shape = (3, img_width, img_height)
else:
  input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

H = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Found 1106 images belonging to 2 classes.
Found 294 images belonging to 2 classes.
Epoch 1/50
18/69 [======>.......................] - ETA: 7:33 - loss: 0.7454 - accuracy: 0.4896

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


37/69 [===============>..............] - ETA: 4:40 - loss: 0.7210 - accuracy: 0.4848

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


69/69 [==============================] - 660s 10s/step - loss: 0.7101 - accuracy: 0.5128 - val_loss: 0.7142 - val_accuracy: 0.5104
Epoch 2/50
69/69 [==============================] - 50s 723ms/step - loss: 0.6676 - accuracy: 0.6183 - val_loss: 0.5652 - val_accuracy: 0.6691
Epoch 3/50
69/69 [==============================] - 49s 707ms/step - loss: 0.5939 - accuracy: 0.7092 - val_loss: 0.5880 - val_accuracy: 0.7014
Epoch 4/50
69/69 [==============================] - 50s 722ms/step - loss: 0.5500 - accuracy: 0.7382 - val_loss: 0.4721 - val_accuracy: 0.7950
Epoch 5/50
69/69 [==============================] - 49s 714ms/step - loss: 0.5149 - accuracy: 0.7621 - val_loss: 0.5313 - val_accuracy: 0.7302
Epoch 6/50
69/69 [==============================] - 48s 694ms/step - loss: 0.4916 - accuracy: 0.7670 - val_loss: 0.5083 - val_accuracy: 0.8129
Epoch 7/50
69/69 [==============================] - 49s 716ms/step - loss: 0.4964 - accuracy: 0.7862 - val_loss: 0.4213 - val_accuracy: 0.7122
Epoch 8/50


In [4]:
import os

model.save(os.path.join(modelPath,'first_try.h5'))

In [5]:
print(train_generator.class_indices)

{'ambulance': 0, 'notambulance': 1}


In [ ]:
test_data_dir = "/content/drive/My Drive/ambulance_prediction/data/test"

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

# dimensions of our images
img_width, img_height = 150, 150

# load the model we saved
model = load_model(os.path.join(modelPath,'first_try.h5'))
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# predicting images
img = image.load_img(os.path.join(test_data_dir,'test3.jpg'), target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict_classes(images, batch_size=10)
print(classes)

[[0]]
